In [1]:
name = 'Payam Taebi'
student_number = '400104867'

In [2]:
#import and load nvcc [using google colab]

!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_yevsg4v
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_yevsg4v
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
directory /content/src already exists
Out bin /content/result.out


#1)
 First, write this program in normal ++Ⅽ̸Ⅽ and calculate the execution time of the program in ⅭPU. If
The dimensions of 10240 were too large and it took too long to run, so run your program in dimensions of 1024.
But consider this issue in your next analysis.

#2)
 Rewrite the program calculations in ⅭUⅮA. At this stage, you do not need to use GPU facilities such as memory shards.

i set size 1024 * 1024 and implement MATRIX MULTIPICATION IN CPU and MATRIX MULTIPICATION IN CUDA USING GPU

In [3]:
%%writefile matrix_mul_cpu_gpu.cu
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <stdbool.h>

#define BLOCK_SIZE 16
int N = 1024;

//implementing C = A * B with CPU
void cpu_mult(int N, float *A, float *B, float *C) {
    float sum_matrix = 0.0f;
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            sum_matrix = 0.0f;
            for (int k = 0; k < N; ++k) {
                sum_matrix += A[N * i + k] * B[N * k + j];
            }
            C[N * i + j] = sum_matrix;
            sum_matrix = 0.0f;
        }
    }
}


// now implementing CUDA mm using GPU
__global__ void gpu_mult(int N, float *A, float *B, float *C) {

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float sum_matrix = 0.0f;
    if (row < N && col < N) {
        for (int i = 0; i < N; i++) {
            sum_matrix = sum_matrix + A[N * row + i] * B[N * i + col];
        }
        C[N * row + col] = sum_matrix;
    }
}


int main(int argc, char const *argv[]) {

    //size of matrix
    int size = sizeof(float) * N * N;
    int matrix_size = N * N;

    // define Matrix
    float *A;
    float *B;
    float *C_cpu;
    float *C_gpu;

    // allocate
    cudaMallocHost((void **) &A, size);
    cudaMallocHost((void **) &B, size);
    cudaMallocHost((void **) &C_cpu, size);
    cudaMallocHost((void **) &C_gpu, size);

    for (int i = 0; i < matrix_size; ++i) {
        A[i] = (float)rand() / RAND_MAX;
        B[i] = (float)rand() / RAND_MAX;
    }

    // now variables initialized and we can calculate result with CPU and GPU

    float gpu_time, cpu_time;

    // initializing temp variables
    float *tA;
    float *tB;
    float *tC;

    cudaMalloc((void **) &tA, size);
    cudaMalloc((void **) &tB, size);
    cudaMalloc((void **) &tC, size);

    // copy var Host to Device
    cudaMemcpy(tA, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(tB, B, size, cudaMemcpyHostToDevice);


    // use cudaEvent_t
    cudaEvent_t start, end;

    // using cudaEventCreate to start the timing
    cudaEventCreate(&start);
    cudaEventCreate(&end);

    // start the record with cudaEventRecord
    cudaEventRecord(start, 0);

    unsigned int grid_rows = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    unsigned int grid_cols = grid_rows;


    // using dm3 in cuda for:
    // dim3 is an integer vector type based on uint3 that is used to specify dimensions.
    // When defining a variable of type dim3, any component left unspecified is initialized to 1.
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);

    // calling gpu mult to start gpu for multiplication
    gpu_mult<<<dimGrid, dimBlock>>> (N, tA, tB, tC);

     // copy device to GPU
    cudaMemcpy(C_gpu, tC, size, cudaMemcpyDeviceToHost);

    // Synchronize
    cudaDeviceSynchronize();

    // finish the record for knowing the time that took
    cudaEventRecord(end, 0);

    // Synchronize
    cudaEventSynchronize(end);

    // blocks execution until the specified event is recorded with cudaEventElapsedTime
    cudaEventElapsedTime(&gpu_time, start, end);



    // now run mult for CPU:
    cudaEventRecord(start, 0);

    cpu_mult(N, A, B, C_cpu);

 // blocks execution until the specified event is recorded with cudaEventElapsedTime and save in cpu_time

    cudaEventRecord(end, 0);
    cudaEventSynchronize(end);
    cudaEventElapsedTime(&cpu_time, start, end);

    // print cpu_time
    printf("Time on CPU: %f seconds\n", cpu_time / 1000);

        // print gpu_time
    printf("Time on GPU: %f seconds\n", gpu_time / 1000);

    // finally, Memory free


    cudaFreeHost(tA);
    cudaFreeHost(tB);
    cudaFreeHost(tC);

    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C_gpu);
    cudaFreeHost(C_cpu);

    return 0;
}


Overwriting matrix_mul_cpu_gpu.cu


In [4]:
!nvcc matrix_mul_cpu_gpu.cu -o matrix_mul_cpu_gpu -lcurand
!./matrix_mul_cpu_gpu

Time on CPU: 8.565032 seconds
Time on GPU: 0.009596 seconds


#3)
 Rewrite the previous program using shared memory. Check that the shared memory is approximately filled in the dimensions of the matrices.

In this section, we have used shared memory to increase speed. With the help of "shared" , we have access to the shared memory. By doing this, the data is copied from the global memory to the shared memory in each block. In this way, the data is closer to the threads and the access time is reduced.
But the result is not much better because we access the matrix elements in the same way as they are stored in the memory.

In [5]:
%%writefile matrix_mul_shared_memory.cu
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <stdbool.h>

#define BLOCK_SIZE 4
#define N 1024


__global__ void gpu_mult_shared(float *A, float *B, float *C) {
    __shared__ float sA[BLOCK_SIZE][N];
    __shared__ float sB[N][BLOCK_SIZE];

    int row = blockIdx.y * BLOCK_SIZE + threadIdx.y;
    int col = blockIdx.x * BLOCK_SIZE + threadIdx.x;

    int e1;
    int e2;
    int t = N / BLOCK_SIZE;

    for (int m = 0; m < t; m++) {
        e1 = m * BLOCK_SIZE + threadIdx.x;
        e2 = m * BLOCK_SIZE + threadIdx.y;
        sA[threadIdx.y][e1] = A[N * row + e1];
        sB[e2][threadIdx.x] = B[N * e2 + col];
    }

    float tmp = 0.0f;

    __syncthreads();

    for (int k = 0; k < N; ++k){
        tmp += sA[threadIdx.y][k] * sB[k][threadIdx.x];
    }
    if (col < N && row < N) C[N * row + col] = tmp;
}

// like before
__global__ void gpu_mult_not_shared(float *A, float *B, float *C) {

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float sum_matrix = 0.0f;
    if (row < N && col < N) {
        for (int i = 0; i < N; i++) {
            sum_matrix = sum_matrix + A[N * row + i] * B[N * i + col];
        }
        C[N * row + col] = sum_matrix;
    }
}

int main(int argc, char const *argv[]) {

    //size of matrix
    int size = sizeof(float) * N * N;
    int matrix_size = N * N;

    // define variables
    float *A;
    float *B;
    float *C_ns;
    float *C_s;

    // allocate
    cudaMallocHost((void **) &A, size);
    cudaMallocHost((void **) &B, size);
    cudaMallocHost((void **) &C_ns, size);
    cudaMallocHost((void **) &C_s, size);

    // filling matrix
    for (int i = 0; i < matrix_size; ++i) {
        A[i] = (float)rand() / RAND_MAX;
        B[i] = (float)rand() / RAND_MAX;
    }


    float gpu_ns_time , gpu_s_time;

    // use cudaEvent_t to know about that
    cudaEvent_t start, end;

    // using cudaEventCreate to start the timing
    cudaEventCreate(&start);
    cudaEventCreate(&end);

    // start the record with cudaEventRecord
    cudaEventRecord(start, 0);

    // define  temp variables
    float *tA;
    float *tB;
    float *tC_ns;
    float *tC_s;

    // allocate
    cudaMalloc((void **) &tA, size);
    cudaMalloc((void **) &tB, size);
    cudaMalloc((void **) &tC_ns, size);
    cudaMalloc((void **) &tC_s, size);

    // copy var Host to Device
    cudaMemcpy(tA, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(tB, B, size, cudaMemcpyHostToDevice);

    unsigned int grid_rows = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    unsigned int grid_cols = grid_rows;

    // using dm3 in cuda for:
    // dim3 is an integer vector type based on uint3 that is used to specify dimensions.
    // When defining a variable of type dim3, any component left unspecified is initialized to 1.
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);

    // calling gpu_mm to start gpu for multiplication
    gpu_mult_not_shared<<<dimGrid, dimBlock>>> (tA, tB, tC_ns);

    // copy device to GPU
    cudaMemcpy(C_ns, tC_ns, size, cudaMemcpyDeviceToHost);

    // Synchronize
    cudaDeviceSynchronize();

    // finish the record for knowing the time that took
    cudaEventRecord(end, 0);

    // Synchronize
    cudaEventSynchronize(end);

    // blocks execution until the specified event is recorded with cudaEventElapsedTime
    cudaEventElapsedTime(&gpu_ns_time, start, end);


    // So far, we have not done anything new and implemented the GPU as in the previous section, and from here on, we will focus on the shared memory section

    // start another start timing:
    cudaEventRecord(start, 0);

    gpu_mult_shared<<<dimGrid, dimBlock>>> (tA, tB, tC_s);

// blocks execution until the specified event is recorded with cudaEventElapsedTime and save in gpu_s_time

    cudaEventRecord(end, 0);
    cudaEventSynchronize(end);
    cudaEventElapsedTime(&gpu_s_time, start, end);

    // print gpu_s_time
    printf("Time on GPU, shared: %f seconds\n", gpu_s_time / 1000);

    // print gpu_time
    printf("Time on GPU, not shared: %f seconds \n", gpu_ns_time / 1000);


    // free memory
    cudaFree(tA);
    cudaFree(tB);
    cudaFree(tC_ns);
    cudaFree(tC_s);

    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C_ns);
    cudaFreeHost(C_s);

    return 0;
}


Overwriting matrix_mul_shared_memory.cu


In [6]:
!nvcc matrix_mul_shared_memory.cu -o matrix_mul_shared_memory -lcurand
!./matrix_mul_shared_memory

Time on GPU, shared: 0.037522 seconds
Time on GPU, not shared: 0.030857 seconds 


#4)
Research the tiling technique. Improve your previous program by using tiling and explain why this program performs better than the previous program.

In this section we use tiling. In this method, we divide the matrix into two tiles and calculate each tile separately. It divides the long access sequence of each thread into phases and uses barrier synchronization to keep the access time of each segment in close time intervals. This technique controls the amount of on-chip memory required by localizing accesses in time and space.

In [7]:
%%writefile tiling.cu

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <stdbool.h>

#define BLOCK_SIZE 32
int N = 10240;

__global__ void gpu_mult_shared(int N, float *A, float *B, float *C) {
    __shared__ float tA[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ float tB[BLOCK_SIZE][BLOCK_SIZE];

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float tmp = 0.0f;

    int m;
    for (int sub = 0; sub < gridDim.x; sub++) {
        m = row * N + sub * BLOCK_SIZE;

        if (N % BLOCK_SIZE != 0)
            tA[threadIdx.y][threadIdx.x] = 0.0f;
        else
            tA[threadIdx.y][threadIdx.x] = A[m];

        m = (sub * BLOCK_SIZE + threadIdx.y) * N + col;

        if (N % BLOCK_SIZE != 0)
            tB[threadIdx.y][threadIdx.x] = 0.0f;
        else
            tB[threadIdx.y][threadIdx.x] = B[m];

        __syncthreads();

        for (int k = 0; k < BLOCK_SIZE; ++k)
            tmp += tA[threadIdx.y][k] * tB[k][threadIdx.x];
        __syncthreads();
    }

    if (col < N && row < N) C[N * row + col] = tmp;
}

__global__ void gpu_mult_not_shared(int N, float *A, float *B, float *C) {
    // like always:
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // A * B = C --> more details in report
    float sum_matrix = 0.0f;
    if (row < N && col < N) {
        for (int i = 0; i < N; i++) {
            sum_matrix = sum_matrix + A[N * row + i] * B[N * i + col];
        }
        C[N * row + col] = sum_matrix;
    }
}

int main(int argc, char const *argv[]) {



    //size of matrix
    int size = sizeof(float) * N * N;
    int matrix_size = N * N;

    // define variables
    float *A;
    float *B;
    float *C_ns;
    float *C_s;

    // allocate
    cudaMallocHost((void **) &A, size);
    cudaMallocHost((void **) &B, size);
    cudaMallocHost((void **) &C_ns, size);
    cudaMallocHost((void **) &C_s, size);

    // filling matrix
    for (int i = 0; i < matrix_size; ++i) {
        A[i] = (float)rand() / RAND_MAX;
        B[i] = (float)rand() / RAND_MAX;
    }


    float gpu_ns_time , gpu_s_time;

    // use cudaEvent_t to know about that
    cudaEvent_t start, end;

    // using cudaEventCreate to start the timing
    cudaEventCreate(&start);
    cudaEventCreate(&end);

    // start the record with cudaEventRecord
    cudaEventRecord(start, 0);

    // define variables
    float *tA;
    float *tB;
    float *tC_ns;
    float *tC_s;

    // allocate
    cudaMalloc((void **) &tA, size);
    cudaMalloc((void **) &tB, size);
    cudaMalloc((void **) &tC_ns, size);
    cudaMalloc((void **) &tC_s, size);

    // copy var Host to Device
    cudaMemcpy(tA, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(tB, B, size, cudaMemcpyHostToDevice);

    unsigned int grid_rows = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    unsigned int grid_cols = grid_rows;

    // using dm3 in cuda for:
    // dim3 is an integer vector type based on uint3 that is used to specify dimensions.
    // When defining a variable of type dim3, any component left unspecified is initialized to 1.
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);

    // calling gpu_mm to start gpu for multiplication

    gpu_mult_not_shared<<<dimGrid, dimBlock>>> (N, tA, tB, tC_ns);

    // copy device to GPU
    cudaMemcpy(C_ns, tC_ns, size, cudaMemcpyDeviceToHost);

    // Synchronize
    cudaDeviceSynchronize();

    // finish the record for knowing the time that took
    cudaEventRecord(end, 0);

    // Synchronize
    cudaEventSynchronize(end);

    // blocks execution until the specified event is recorded with cudaEventElapsedTime
    cudaEventElapsedTime(&gpu_ns_time, start, end);



    //start cpu and start timing:
    cudaEventRecord(start, 0);

    // calling the function
    gpu_mult_shared<<<dimGrid, dimBlock>>> (N, tA, tB, tC_s);


 // blocks execution until the specified event is recorded with cudaEventElapsedTime and save in gpu_s_time

    cudaEventRecord(end, 0);
    cudaEventSynchronize(end);
    cudaEventElapsedTime(&gpu_s_time, start, end);

    printf("for n = %d :\n",N);

    // print shared
    printf("Time on GPU, shared: %f seconds\n", gpu_s_time / 1000);

    // print not shared
    printf("Time on GPU, not shared: %f seconds\n", gpu_ns_time / 1000 );

    cudaFree(tA);
    cudaFree(tB);
    cudaFree(tC_ns);
    cudaFree(tC_s);

    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C_ns);
    cudaFreeHost(C_s);

    return 0;
}


Overwriting tiling.cu


In [8]:
!nvcc tiling.cu -o tiling -lcurand
!./tiling

for n = 10240 :
Time on GPU, shared: 2.361471 seconds
Time on GPU, not shared: 3.722106 seconds


#5)
Research memory coalescing on GPU. Now which LOAD commands in your program use MEMORY COALESCING and which ones don't? What method do you suggest for using MEMORY COALESCING in these commands? Implement it and report the implementation OVERHEAD as well as the resulting performance improvement.

Storing matrices as a contiguous n * n array takes advantage of the computer's architecture, as reading an element of the matrix will also make its row neighbors accessible in the cache. Generally, reading a row is much faster than reading a column. In the program, whenever we read elements of the first matrix, we are actually using memory coalescing because we are reading from the row.

Now, in order to use this method for reading columns as well and make the calculations faster, we need to transpose the second matrix. This way, to calculate the element i,j, we will need to multiply row i of the first matrix with row j of the transposed second matrix. I have implemented this change in the multiplication using shared memory, but naturally, there is an additional overhead for calculating the transpose of the matrix.

In [9]:
%%writefile mult_transpose.cu
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <stdbool.h>

#define BLOCK_SIZE 4
#define N 1024


__global__ void gpu_transpose(float *input, float *output) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    output[x * N + y] = input[y * N + x];
}

__global__ void gpu_mult_shared_transpose(float *A, float *B, float *C) {

    __shared__ float sA[BLOCK_SIZE][N];
    __shared__ float sB[BLOCK_SIZE][N];

    int row = blockIdx.y * BLOCK_SIZE + threadIdx.y;
    int col = blockIdx.x * BLOCK_SIZE + threadIdx.x;

    int e1;
    int e2;
    int t = N / BLOCK_SIZE;

    for (int m = 0; m < t; m++) {
        e1 = m * BLOCK_SIZE + threadIdx.x;
        e2 = m * BLOCK_SIZE + threadIdx.y;
        sA[threadIdx.y][e1] = A[N * row + e1];
        sB[threadIdx.x][e2] = B[N * col + e2];
    }

    float tmp = 0.0f;

    __syncthreads();

    for (int k = 0; k < N; ++k){
        tmp += sA[threadIdx.y][k] * sB[threadIdx.x][k];
    }
    if (col < N && row < N) C[N * row + col] = tmp;
}

// like before
__global__ void gpu_mult_not_shared(float *A, float *B, float *C) {

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    float sum_matrix = 0.0f;
    if (row < N && col < N) {
        for (int i = 0; i < N; i++) {
            sum_matrix = sum_matrix + A[N * row + i] * B[N * i + col];
        }
        C[N * row + col] = sum_matrix;
    }
}

int main(int argc, char const *argv[]) {

    //size of matrix
    int size = sizeof(float) * N * N;
    int matrix_size = N * N;

    // define variables
    float *A;
    float *B;
    float *C_ns;
    float *C_s;

    // allocate
    cudaMallocHost((void **) &A, size);
    cudaMallocHost((void **) &B, size);
    cudaMallocHost((void **) &C_ns, size);
    cudaMallocHost((void **) &C_s, size);

    // filling matrix
    for (int i = 0; i < matrix_size; ++i) {
        A[i] = (float)rand() / RAND_MAX;
        B[i] = (float)rand() / RAND_MAX;
    }


    float gpu_ns_time , gpu_s_time;

    // use cudaEvent_t to know about that
    cudaEvent_t start, end;

    // using cudaEventCreate to start the timing
    cudaEventCreate(&start);
    cudaEventCreate(&end);

    // start the record with cudaEventRecord
    cudaEventRecord(start, 0);

    // define  temp variables
    float *tA;
    float *tB;
    float *tBT;
    float *tC_ns;
    float *tC_s;

    // allocate
    cudaMalloc((void **) &tA, size);
    cudaMalloc((void **) &tB, size);
    cudaMalloc((void **) &tBT, size);
    cudaMalloc((void **) &tC_ns, size);
    cudaMalloc((void **) &tC_s, size);

    // copy var Host to Device
    cudaMemcpy(tA, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(tB, B, size, cudaMemcpyHostToDevice);

    unsigned int grid_rows = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;
    unsigned int grid_cols = grid_rows;

    // using dm3 in cuda for:
    // dim3 is an integer vector type based on uint3 that is used to specify dimensions.
    // When defining a variable of type dim3, any component left unspecified is initialized to 1.
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);

    //calculate gpu B_transpose
    gpu_transpose<<<dimGrid, dimBlock>>>(tB, tBT);

    // calling gpu_mm to start gpu for multiplication
    gpu_mult_not_shared<<<dimGrid, dimBlock>>> (tA, tB, tC_ns);

    // copy device to GPU
    cudaMemcpy(C_ns, tC_ns, size, cudaMemcpyDeviceToHost);

    // Synchronize
    cudaDeviceSynchronize();

    // finish the record for knowing the time that took
    cudaEventRecord(end, 0);

    // Synchronize
    cudaEventSynchronize(end);

    // blocks execution until the specified event is recorded with cudaEventElapsedTime
    cudaEventElapsedTime(&gpu_ns_time, start, end);


    // So far, we have not done anything new and implemented the GPU as in the previous section, and from here on, we will focus on the shared memory section

    // start another start timing:
    cudaEventRecord(start, 0);

    gpu_mult_shared_transpose<<<dimGrid, dimBlock>>> (tA, tBT, tC_s);

// blocks execution until the specified event is recorded with cudaEventElapsedTime and save in gpu_s_time

    cudaEventRecord(end, 0);
    cudaEventSynchronize(end);
    cudaEventElapsedTime(&gpu_s_time, start, end);

    // print gpu_s_time
    printf("Time on GPU, shared using B^T instead of B: %f seconds\n", gpu_s_time / 1000);

    // print gpu_time
    printf("Time on GPU, not shared: %f seconds \n", gpu_ns_time / 1000);


    // free memory
    cudaFree(tA);
    cudaFree(tB);
    cudaFree(tC_ns);
    cudaFree(tC_s);

    cudaFreeHost(A);
    cudaFreeHost(B);
    cudaFreeHost(C_ns);
    cudaFreeHost(C_s);

    return 0;
}


Overwriting mult_transpose.cu


In [10]:
!nvcc mult_transpose.cu -o mult_transpose -lcurand
!./mult_transpose

Time on GPU, shared using B^T instead of B: 0.040667 seconds
Time on GPU, not shared: 0.031029 seconds 
